In [72]:
using Gurobi, JuMP
using CSV, DataFrames
using LinearAlgebra, Random, Statistics
using PyPlot
Random.seed!(15095); # Set seed for reproducibility

In [97]:
data = CSV.read("data/final_data.csv", DataFrame);
train = data[data.Year .< 2014, :]
test = data[data.Year .>= 2014, :]
test = dropmissing(test)
lnr = IAI.OptKNNImputationLearner()
train = IAI.fit_transform!(lnr, train)

X_train, y_train = Matrix(select(train, Not(["Tm", "Year", "WinLoss"]))), train[:, "WinLoss"]
X_test, y_test = Matrix(select(test, Not(["Tm", "Year", "WinLoss"]))), test[:, "WinLoss"];

In [118]:
features = names(select(train, Not(["Tm", "Year", "WinLoss"])))
features_transformed = [transf * f for f in features for transf in ["", "squared_", "sqrt_", "exp_", "log_"]];

In [119]:
X_train_norm = (X_train .- minimum(vcat(X_train, X_test), dims = 1)) ./ (maximum(vcat(X_train, X_test), dims = 1) - minimum(vcat(X_train, X_test), dims = 1))
X_test_norm = (X_test .- minimum(vcat(X_train, X_test), dims = 1)) ./ (maximum(vcat(X_train, X_test), dims = 1) - minimum(vcat(X_train, X_test), dims = 1));

In [120]:
# Create a function that transforms the matrix input matrix by adding non-linear transformations to all columns
function transform_data(X, eps = 0.1)

    n, p = size(X) # Store the matrix dimension
    # Initiate the transformed matrix with the first feature
    Xt = hcat(X[:,1], X[:,1].^2, X[:,1].^0.5, exp.(X[:,1]), log.(X[:,1] .+ eps))

    # Loop for all other features and concatenate the transformations to the matrix
    for i = 2:p
        Xt = hcat(Xt, X[:,i], X[:,i].^2, X[:,i].^0.5, exp.(X[:,i]), log.(X[:,i] .+ eps))
    end

    return Xt;

end;

In [121]:
# Create a function that returns a list of feature couples whose correlation exceeds rho
function pairwise_correlation(X, rho)

    _, p = size(X) # Store the number of features
    cor_matrix = cor(X); # Compute the correlation matrix
    cor_variables = [] # Initialize an empty list to store all pairwise correlated features couples

    for i = 1:p 
        for j = 1:i-1 # Loop under all values of the lower diagonal of the correlation matrix

            if abs(cor_matrix[i,j]) > rho
                # If the correlation coefficient of features i and j exceeds rho, append the couple to the list
                push!(cor_variables, (i,j))
            end
            
        end
    end

    return cor_variables;

end;

In [143]:
function holistic_regression(X, y, rho, lambda, k, M)

    # Create the transformed matrix
    X_transformed = transform_data(X);
    n, p = size(X) # Store the input matrix size
    _, p_tilde = size(X_transformed) # Set the number of features in the transformed matrix
    augmentation = Int(p_tilde / p)
    # Compute the list of correlated feature couples
    HC = pairwise_correlation(X_transformed, rho)

    # Create JuMP model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Introduce model variables
    @variable(model, Beta[1:p_tilde])
    @variable(model, a[1:p_tilde] >= 0)
    @variable(model, z[1:p_tilde], binary = true)

    # Robustness term constraint linearization
    @constraint(model,[j = 1:p_tilde], Beta[j] <= a[j])
    @constraint(model,[j = 1:p_tilde], -Beta[j] <= a[j])

    # Big-M integer constraint for sparsity
    @constraint(model,[j = 1:p_tilde], Beta[j] <= M*z[j])
    @constraint(model,[j = 1:p_tilde], Beta[j] >= -M*z[j])
    # Sparsity constraint
    @constraint(model, sum(z[j] for j = 1:p_tilde) <= k)

    # Non-linear transformations constraint
    @constraint(model, [j = 1:p], sum(z[augmentation*(j-1)+i] for i = 1:augmentation) <= 1)

    # Pairwise collinearity
    @constraint(model, [i = 1:length(HC)], z[HC[i][1]] + z[HC[i][2]] <= 1)

    # Implement the objective function of the problem
    @objective(model, Min, sum((y[i] - sum(X_transformed[i,j]*Beta[j] for j=1:p_tilde))^2 for i = 1:n) + lambda * sum(a[j] for j = 1:p_tilde))

    # Solve the optimization problem
    optimize!(model);

    Beta, z = JuMP.value.(Beta), Int.(round.(JuMP.value.(z)))

    y_pred = X_transformed * Beta
    r2 = 1 - sum((y_pred .- y).^2) / sum((0.5 .- y).^2)
    print("Training R2: $(r2)")

    return Beta, z, r2

end;

In [150]:
# Specify the holisitic regression problem parameters
rho = 0.7
lambda = 0.01
k = 10
M = 100

# Compute the holistic regression model
Beta, z, _ = holistic_regression(X_train_norm, y_train, rho, lambda, k, M)
X_test_transformed = transform_data(X_test_norm);
y_pred = X_test_transformed * Beta;
r2 = 1 - sum((y_pred .- y_test).^2) / sum((0.5 .- y_test).^2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Training R2: 0.5419875935374558

0.4281358608642384

In [151]:
for zidx in findall(z .== 1)
    println(features_transformed[zidx], " :  ", Beta[zidx])
end

log_PTS_1 :  0.11896170343603811
exp_PER_1 :  -0.033132947448919745
exp_VORP_2 :  0.15276310405270352
exp_PER_2 :  -0.10649886279099867
squared_VORP_3 :  0.09487749003455642
log_SRS_2 :  -0.041901136332918844
exp_SRS_1 :  0.29046405233427147
log_Rest :  -0.021893815451698922
log_distLB :  0.018561385424663183
squared_distUB :  -0.04668325465148927
